In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glasses.nn.models.classification.fishnet import *
import torch
import torch.nn as nn

In [11]:
x = torch.rand((1,3,224,224))

FishNetEncoder(3)(x).shape

Tail Channels :  [(64, 128), (128, 256), (256, 512)]
Body Channels :  [(512, 512), (768, 384), (512, 256)]
Head Channels :  [(320, 320), (832, 832), (1600, 1600), (2112, 2112)]
torch.Size([1, 256, 14, 14])
torch.Size([1, 128, 28, 28])
torch.Size([1, 64, 56, 56])
[torch.Size([1, 768, 14, 14]), torch.Size([1, 512, 28, 28]), torch.Size([1, 320, 56, 56])]


RuntimeError: Given groups=1, weight of size [64, 256, 1, 1], expected input[1, 320, 56, 56] to have 256 channels, but got 320 channels instead

In [10]:
FishNetEncoder(3).head

Tail Channels :  [(64, 128), (128, 256), (256, 512)]
Body Channels :  [(512, 512), (768, 384), (512, 256)]
Head Channels :  [(320, 320), (832, 832), (1600, 1600), (2112, 2112)]


ModuleList(
  (0): FishNetURBlock(
    (transfer): FishNetBottleNeck(
      (block): Sequential(
        (0): ConvBnAct(
          (conv): Conv2dPad(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (1): ConvBnAct(
          (conv): Conv2dPad(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
        (2): ConvBnAct(
          (conv): Conv2dPad(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(inplace=True)
        )
      )
      (act): ReLU(inplace=True)
    )
    (block): Sequential(
      (0): FishNetBottleNeck(
        (block): Sequential(
       

In [ ]:
x= torch.rand((1,512,7,7))
trans_x = torch.rand((1,512, 14, 14))

block = FishNetURBlock(512, 512)
# print(block)
block(x, trans_x).shape

In [ ]:
FishNetURBlock(512, 256)

In [ ]:
num_blk = 3
start_c = 64
tail_channels = [(start_c, start_c*2)]
for i in range(num_blk - 1):
    tail_channels.append((tail_channels[-1][1], tail_channels[-1][1] * 2))
print("Tail Channels : ", tail_channels)

in_c, transfer_c = tail_channels[-1][1], tail_channels[-2][1]
body_channels = [(in_c, in_c), (in_c + transfer_c, (in_c + transfer_c)//2)]
# First body module is not change feature map channel
for i in range(1, num_blk-1):
    transfer_c = tail_channels[-i-2][1]
    in_c = body_channels[-1][1] + transfer_c
    body_channels.append((in_c, in_c//2))
print("Body Channels : ", body_channels)

in_c = body_channels[-1][1] + tail_channels[0][0]
head_channels = [(in_c, in_c)]
for i in range(num_blk):
    transfer_c = body_channels[-i-1][0]
    in_c = head_channels[-1][1] + transfer_c
    head_channels.append((in_c, in_c))
print("Head Channels : ", head_channels)

[(512, 512), (768, 384), (512, 256)]


In [ ]:
import numpy as np



In [ ]:
FishNetTail(64)(torch.rand((1,64, 56, 56))).shape

In [ ]:
class GrowModuleList(nn.ModuleList):
    def __init__(self, block: nn.Module, start_features: int = 64, n: int = 4, *args, **kwargs):
        widths = [start_features]
        for _ in range(n):
            widths.append(widths[-1] * 2)
        self.in_out_widths = list(zip(widths, widths[1:]))
        blocks = [block(in_f, out_f, *args, **kwargs) for in_f, out_f in self.in_out_widths]
        super().__init__(blocks)

In [ ]:
GrowModuleList(nn.Conv2d, kernel_size=3)